In [1]:
import os
import gc
import argparse
import json
import random
import math
import random
from functools import reduce
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split, ShuffleSplit, StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_recall_fscore_support, classification_report
import torch
from torch import nn
from torch.optim import Adam, SGD, AdamW
from torch.nn import functional as F
from performer_pytorch import PerformerLM
import scanpy as sc
import anndata as ad
from utils import *
import pickle as pkl

In [2]:
CLASS = 7
SEED = 2021
EPOCHS = 100
SEQ_LEN = 16906 + 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
data_path = 'data/preprocessed_data.h5ad'
data = sc.read_h5ad(data_path)
data = data.X
print(type(data))
data.shape

<class 'scipy.sparse._csr.csr_matrix'>


(5, 16906)

In [9]:
model = PerformerLM(
    num_tokens = 50,
    dim = 200,
    depth = 6,
    max_seq_len = SEQ_LEN,
    heads = 10,
    local_attn_heads = 0,
    g2v_position_emb = True
)

model_path = 'model/scbert_pretrain.pth'
# ckpt = torch.load(model_path, map_location=torch.device('cpu'))
# model.load_state_dict(ckpt['model_state_dict'])
for param in model.parameters():
    param.requires_grad = False
# model = model.to(device)

/home/agauthier/scbert/performer_pytorch/performer_pytorch.py:115: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:2485.)
  q, r = torch.qr(unstructured_block.cpu(), some = True)


In [ ]:
batch_size = data.shape[0]
model.eval()
pred_finals = []
novel_indices = []
with torch.no_grad():
    for index in range(batch_size):
        full_seq = data[index].toarray()[0]
        full_seq[full_seq > (CLASS - 2)] = CLASS - 2
        full_seq = torch.from_numpy(full_seq).long()
        full_seq = torch.cat((full_seq, torch.tensor([0])))#.to(device)
        full_seq = full_seq.unsqueeze(0)
        print(full_seq)
        embeddings = model(full_seq, return_encodings = True)
        print('emebd:', embeddings)
        pred_finals.append(embeddings)
# print(pred_finals)

tensor([[-9223372036854775808,                    0,                    0,
          ..., -9223372036854775808, -9223372036854775808,
                            0]])


IndexError: index out of range in self